# 畳み込みニューラルネットワークの実装

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import sys,os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1,3,7,7)
col1 = im2col(x1,5,5,stride = 1,pad=0)
print(col1.shape)

x2 = np.random.rand(10,3,7,7)
col2 = im2col(x2,5,5,stride = 1,pad=0)
print(col2.shape)

x3 = np.random.rand(100,3,8,8)
col3 = im2col(x3,5,5,stride=1,pad=0)
print(col3.shape)

(9, 75)
(90, 75)
(1600, 75)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのモジュールを利用可能にするための設定
from common.util import im2col  # im2colをインポート（畳み込みを効率化する関数）

# Convolution（畳み込み層）のクラス定義
class convolution:
    def __init__(self, w, b, stride=1, pad=0):
        """
        コンストラクタ（初期化）
        w: 重み（フィルタ）→ 形状は (フィルタ数, 入力チャネル数, フィルタ高さ, フィルタ幅)
        b: バイアス → 形状は (フィルタ数,)
        stride: ストライド（フィルタをスライドする間隔）
        pad: パディング（入力データの周囲に付加するゼロの数）
        """
        self.w = w  # 重み（フィルタ）
        self.b = b  # バイアス
        self.stride = stride  # ストライド
        self.pad = pad  # パディング

    def forward(self, x):
        """
        順伝播（Forward）
        x: 入力データ → 形状は (バッチ数, 入力チャネル数, 高さ, 幅)
        """
        # 畳み込み後の出力の形状を計算
        FN, C, FH, FW = self.w.shape  # フィルタの形状（フィルタ数、チャネル数、高さ、幅）
        N, C, H, W = x.shape  # 入力データの形状（バッチ数、チャネル数、高さ、幅）
        out_h = int(1 + (H + 2 * self.pad - FH) / self.stride)  # 出力の高さ
        out_w = int(1 + (W + 2 * self.pad - FW) / self.stride)  # 出力の幅

        # 入力データをim2colで展開（パッチ化）
        col = im2col(x, FH, FW, self.stride, self.pad)
        # フィルタをim2colに対応する形状に変換
        col_w = self.w.reshape(FN, -1).T  # フィルタを2次元配列に変換

        # 行列積を計算（展開された入力データとフィルタ）
        out = np.dot(col, col_w) + self.b  # 畳み込み演算

        # 出力の形状を元に戻す（reshapeしてtranspose）
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)  # (バッチ数, フィルタ数, 出力高さ, 出力幅)
        return out

class Pooling:
    def __init__(self,pool_h,pool_w,stride=1,pad=0):
        self.pool_h=pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
    def forward(self,x):
        N,C,H,W=x.shape
        out_h=int(1+(H-self.pool_h)/self.stride)
        out_w=int(1+(W-self.pool_w)/self.stride)
        col = im2col(x,self.pool_h,self.pool_w,self.stride,self.pad)
        col = col.reshape(-1,self.pool_h*self.pool_w)

        out = np.max(col,axis=1)
        out = out.reshape(N,out_h,out_w,C).transpose(0,3,1,2)
        print(out.shape)
        return out
